# Exploración y Análisis de Datos (EDA)

[![Banner.png](https://i.postimg.cc/C1t2SCwf/Banner.png)](https://postimg.cc/FdypxLd9)

## Introducción

Este notebook se centra en el análisis exploratorio de datos (EDA) para comprender mejor las características de las transacciones financieras y detectar patrones relacionados con el fraude. Las tareas principales incluyen:

**Exploración descriptiva:** Examinar las características de los datos, como la distribución de variables, relaciones entre variables y valores atípicos.

**Visualización de datos:** Crear gráficos y tablas para visualizar la distribución de los datos, identificar patrones y tendencias, y destacar posibles indicadores de fraude.

**Análisis de correlación:** Evaluaremos la relación entre variables para comprender cómo se asocian los diferentes factores con el fraude.

**Identificación de características relevantes:** Seleccionar las variables más importantes para el análisis posterior, como el monto de la transacción, el método de pago y la ubicación del cliente.
Objetivos:

Obtener una comprensión profunda de las características de las transacciones financieras.
Identificar patrones y tendencias que puedan indicar actividades fraudulentas.
Seleccionar las variables más relevantes para la construcción de modelos de predicción de fraude.


In [ ]:
# aquí cargaremos los df a trabajar